In [14]:
import logging
logging.basicConfig(level=logging.INFO)
from pyrosetta import *
import pyrosetta.toolbox

import matplotlib.pyplot as plt
import numpy as np

pyrosetta.init("-ex1 -ex2aro")

INFO:pyrosetta.rosetta:Found rosetta database at: /usr/local/lib/python3.8/dist-packages/pyrosetta-2022.5+release.b3365a12916-py3.8-linux-x86_64.egg/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2022 [Rosetta PyRosetta4.Release.python38.ubuntu 2022.05+release.b3365a12916d9c125d41cdee36639019f73be202 2022-02-03T16:09:08] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python38.ubuntu r310 2022.05+release.b3365a12916 b3365a12916d9c125d41cdee36639019f73be202 http://www.pyrosetta.org 2022-02-03T16:09:08
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.8/dist-packages/pyrosetta-2022.5+release.b3365a12916-py3.8-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG d

PyRosetta-4 2022 [Rosetta PyRosetta4.Release.python38.ubuntu 2022.05+release.b3365a12916d9c125d41cdee36639019f73be202 2022-02-03T16:09:08] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [22]:
#initialize the pose and the score function

start_pose = pose_from_pdb("selicase_form1.pdb")
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

INFO:rosetta:core.import_pose.import_pose: File 'selicase_form1.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LYS:CtermProteinFull 108
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Initializing IdealParametersDatabase with default Ks=300 , 80 , 80 , 10 , 80
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-lengths.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Read 759 bb-independent lengths.
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-angles.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Read 1434 bb-independent angles.
INFO:rosetta:basic.io.database: Database file opened: scoring/score_functions/bondlength_bondangle/default-torsions.txt
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Read 1 bb-independent torsions.
INFO:rosetta:ba

In [23]:
#create task factory for FastDesign
#resfile is included
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile("resfile"))

# Convert the task factory into a PackerTask to take a look at it
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

INFO:rosetta:core.pack.task: Packer task: initialize from command line()


#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
6	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
7	TRUE	TRUE	ALA

In [24]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True) #allowing all movements
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [25]:
#set up the fast relax movcer
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone")

In [26]:
### BEGIN SOLUTION
#apply fast relax
%time fr.apply(pose)

INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (108)
INFO:rosetta:protocols.relax.FastRelax: CMD: repeat  -100.161  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: coord_cst_weight  -100.161  0  0  0.55
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  -139.914  0  0  0.022
INFO:rosetta:core.pack.task: Packer task: initialize from command line()
INFO:rosetta:core.pack.pack_rotamers: built 8191 rotamers at 108 positions.
INFO:rosetta:core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Adding undefined angle PRO: CD,N,1H to DB with theta0 = 0 , Ktheta = 0
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Adding undefined angle PRO: CD,N,2H to DB with theta0 = 0 , Ktheta = 0
INFO:rosetta:protocols.relax.FastRelax: CMD: repack  -574.118  0  0  0.022
INFO:rosetta:protocols.relax.FastRelax: CMD: scale:fa_rep  -567.589  0  0  

CPU times: user 7min 14s, sys: 2.41 s, total: 7min 16s
Wall time: 7min 26s


In [29]:
#score the designed pose and dump the pdb
scorefxn(pose)
pose.dump_pdb("mutaselecase1.pdb")

True

In [30]:
#get the new sequence to put into alphafold
pose.sequence()

'FEEEEDRKRHEEKKKEMKLNDKKANIKIKIKPLKRKIASISLTNKTIYINKNILPYLSDEEIRFILAHELLHLKYGKYHINEFEEELLFLFPNKEAILINLINKLHQK'

In [33]:
#score the alphafold pose with the same structure
pose2 = pose_from_pdb("test_8d3d8_unrelaxed_rank_1_model_5.pdb")
scorefxn(pose2)

INFO:rosetta:core.import_pose.import_pose: File 'test_8d3d8_unrelaxed_rank_1_model_5.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LYS:CtermProteinFull 108
INFO:rosetta:core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (108)


581.9679398064809